In [1]:
from Bio import SeqIO,SeqFeature,Entrez
import xml.etree.ElementTree as ET
import pandas as pd
import time
import os

Based on the two provided sample files I suposse that the potential customer would like to integrate genomic data directly with the biological literature.  The connection between provided [GeneBank file format](https://www.ncbi.nlm.nih.gov/Sitemap/samplerecord.html) which originates from NCBI Nucleotide database and xml file which represents  [Pubmed article metadata](https://www.nlm.nih.gov/bsd/licensee/elements_descriptions.html) could be established trough PubMed ID. The main two entities in this case would be Gene and Scientific Publication. 

 Open the sequence.gb file. Collect the record name and inspect the description.

In [83]:
for index, record in enumerate(SeqIO.parse((os.path.join("./Data/","sequence.gb")), "genbank")):
    print(f'''Index: {index}\nRecordName: {record.name}\nRecordDescription: {record.description}''')
         

Index: 0
RecordName: NG_015888
RecordDescription: Homo sapiens cyclin dependent kinase 6 (CDK6), RefSeqGene (LRG_991) on chromosome 7


Collect the following:
- **Gene Symbol**  ("The official gene symbol approved by the HGNC, which is typically a short form of the gene name. Symbols are approved in accordance with the Guidelines for Human Gene Nomenclature."),
- **Gene Name**  ("The full gene name approved by the HGNC; corresponds to the approved symbol above."), 
- **Gene Synonyms**  ("Alternative symbols that have been used to refer to the gene. Aliases may be from literature, from other databases or may be added to represent membership of a gene group."). 


In [6]:
for x in record.features:
    if (x.type == 'gene') and (x.qualifiers['gene'][0]=='CDK6'):
        db_xref = x.qualifiers['db_xref']
        print(f'''GeneSymbol: {x.qualifiers['gene'][0]}\nGeneSynonyms: {x.qualifiers['gene_synonym'][0]}\nGeneName: {x.qualifiers['note'][0]}''')
        

GeneSymbol: CDK6
GeneSynonyms: MCPH12; PLSTIRE
GeneName: cyclin dependent kinase 6


Collect the  following:
- **Chromosome** (Indicates the chromosome number.) and **Cytogenetic band** (Indicates the cytogenetic location.) 
- **Organism** and **Taxon ID** that it originates from.

In [13]:
for x in record.features:
    if x.type == 'source':
        print(f'''Chromosome: {x.qualifiers['chromosome'][0]}\nCytogeneticBand: {x.qualifiers['map'][0]}\nOrganism: {x.qualifiers['organism'][0]}\nTaxonID: {x.qualifiers['db_xref'][0].split(':')[1]}''') 


Chromosome: 7
CytogeneticBand: 7q21.2
Organism: Homo sapiens
TaxonID: 9606


Collect the [HUGO Gene Nomenclature Committee](https://www.genenames.org/), 
[Online Mendelian Inheritance in Man](https://www.omim.org/) and [NCBI Gene](https://www.ncbi.nlm.nih.gov/gene/) identifiers.
Note: If a customer has a licence to use OMIM I would include those links also, since it would be easy to enrich the associations with gene and disease or disorder links.

In [84]:
for x in record.features:
    if (x.type == 'gene') and (x.qualifiers['gene'][0]=='CDK6'):
        db_xref = x.qualifiers['db_xref']
        for r in db_xref:
            arr = r.split(':')
            key = arr[0]
            value = ':'.join(arr[1:])
            print(f'''{key}: {value}''')
            

GeneID: 1021
HGNC: HGNC:1777
MIM: 603368


Collect the [Reference Sequence](https://www.ncbi.nlm.nih.gov/refseq/) transcript IDs. 
**Note**: The sequence NM_001259.6 has been updeated; the current version is NM_001259.7.

In [17]:
for  x in record.features:
    if (x.type == 'mRNA')and (x.qualifiers['gene'][0]=='CDK6'):
        print(f'''RefSeqTranscriptID: {x.qualifiers['transcript_id'][0]}''')
        

RefSeqTranscriptID: NM_001145306.1
RefSeqTranscriptID: NM_001259.6


Collect  [Reference Sequence](https://www.ncbi.nlm.nih.gov/refseq/) protein IDs and [Consensus CDS sequence](https://www.ncbi.nlm.nih.gov/CCDS/CcdsBrowse.cgi) ID.

In [18]:
for x in record.features:
    if (x.type == 'CDS')and (x.qualifiers['gene'][0]=='CDK6'):
        
        db_xref = x.qualifiers['db_xref']
        for r in db_xref:
            if 'CCDS' in r:
                key, val = r.split(":")
                print(key,val)
        print(f'''RefSeqProteinID: {x.qualifiers['protein_id'][0]} ''')       
        
            

CCDS CCDS5628.1
RefSeqProteinID: NP_001138778.1 
CCDS CCDS5628.1
RefSeqProteinID: NP_001250.1 


Extract information related to publication such as Title, PubMed ID, Journal, Authors in order to compare with the information from the publication metadata file.

In [20]:
for p in record.annotations['references']:
     print(f'''Title: {p.title}\nPubMedID: {p.pubmed_id}\nAuthors: {p.authors}\nJournal: {p.journal}''')    
    

Title: Primary Autosomal Recessive Microcephalies and Seckel Syndrome Spectrum Disorders
PubMedID: 20301772
Authors: Verloes,A., Drunat,S., Gressens,P. and Passemard,S.
Journal: (in) Adam MP, Ardinger HH, Pagon RA, Wallace SE, Bean LJH, Stephens K and Amemiya A (Eds.); GENEREVIEWS((R)); (1993)


Process  PubMed metadata file. Extract properties relevat to scientific publication.

In [29]:
tree = ET.parse(os.path.join('./Data/','pubmed_result.xml'))
root = tree.getroot()

Collect PubmedID.

In [85]:
for x in root.findall(".//PMID"):
    print(f'''PubMedID: {x.text}''')

PubMedID: 20301772


Collect Publisher Name.

In [86]:
for x in root.findall(".//PublisherName"):
    print(f'''Publisher: {x.text}''')

Publisher: University of Washington, Seattle


Collect Journal Name.

In [87]:
for x in root.findall(".//BookTitle"):
    print(f'''Journal: {x.text}''')

Journal: GeneReviews


Collect Year of publication.

In [36]:
for x in root.findall(".//PubDate/Year"):
    print(f'''PublicationYear: {x.text}''')

PublicationYear: 1993


Collect the Title of Publication.

In [88]:
for x in root.findall(".//ArticleTitle"):
    print(f'''Title: {x.text}''')

Title: Primary Autosomal Recessive Microcephalies and Seckel Syndrome Spectrum Disorders


Collect the publication Language.

In [89]:
for x in root.findall(".//Language"):
    print(f'''Language: {x.text}''')

Language: eng


Collect the publication type.

In [40]:
for x in root.findall(".//PublicationType"):
    print(f'''PyblicationType: {x.text}''')

PyblicationType: Review


Collect the list of Authors.

In [99]:
for x in root.findall(".//AuthorList"):
    if x.get('Type') == 'authors':
        for e in list(x):
            print(f'''Author: {e.find('./LastName').text} {e.find('./ForeName').text}''')

Author: Verloes Alain
Author: Drunat Séverine
Author: Gressens Pierre
Author: Passemard Sandrine


#### Find all papers related to CDK6 gene.
I have used  curated gene-PMID links from the [Entrez Gene database](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3013746/) to download [gene2pubmed](ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2pubmed.gz) tsv file from 12th April 2019 which has the links between genes and publications over three columns, denoting NCBI taxonomy ID, Entrez Gene ID and PMID. This gene has been mentioned in 311 publications. 

In [96]:
gene2pubmed = pd.read_csv("./Data/gene2pubmed.txt"
                              ,sep= "\t",index_col= False).rename(str.lower,axis = 'columns')

In [97]:
cdk6_related_papers = gene2pubmed[gene2pubmed['geneid'] == 1021]

In [98]:
cdk6_related_papers.shape

(311, 3)

In [92]:
genes_related_to_paper = gene2pubmed[gene2pubmed['pubmed_id'] ==20301772]

**Note:** Please add your email adress in order for this function to work.

In [93]:
def fetch_gene_names (identifier):
    Entrez.email = "email@domain.com"
    handle = Entrez.epost('gene', id = identifier)
    results = Entrez.read(handle)
    webEnv = results["WebEnv"]
    queryKey = results["QueryKey"]
    time.sleep(0.5)
    data = Entrez.efetch(db="gene", webenv=webEnv, query_key =
                queryKey, retmode="xml")
    annotations = Entrez.read(data)
    gene_names = {}
    for annotation in annotations:
        for property in annotation['Entrezgene_properties']:
             if property.get("Gene-commentary_label",None) == "Nomenclature":
                    for subproperty in property["Gene-commentary_properties"]:
                        if subproperty.get("Gene-commentary_label",None)  == "Official Symbol":
                            gene_names['symbol'] =subproperty.get("Gene-commentary_text","")
                        if subproperty.get("Gene-commentary_label",None)  == "Official Full Name":
                            gene_names['full_name'] =subproperty.get("Gene-commentary_text","")
    return gene_names


In [94]:
genes = list(genes_related_to_paper['geneid'])

In [95]:
for gene in genes:
    try:
        print(fetch_gene_names(gene))
    except:
        time.sleep(1)
        print(fetch_gene_names(gene))

{'symbol': 'ATR', 'full_name': 'ATR serine/threonine kinase'}
{'symbol': 'CDK6', 'full_name': 'cyclin dependent kinase 6'}
{'symbol': 'PHC1', 'full_name': 'polyhomeotic homolog 1'}
{'symbol': 'RBBP8', 'full_name': 'RB binding protein 8, endonuclease'}
{'symbol': 'STIL', 'full_name': 'STIL centriolar assembly protein'}
{'symbol': 'CEP135', 'full_name': 'centrosomal protein 135'}
{'symbol': 'CEP152', 'full_name': 'centrosomal protein 152'}
{'symbol': 'NIN', 'full_name': 'ninein'}
{'symbol': 'CDK5RAP2', 'full_name': 'CDK5 regulatory subunit associated protein 2'}
{'symbol': 'CENPJ', 'full_name': 'centromere protein J'}
{'symbol': 'KNL1', 'full_name': 'kinetochore scaffold 1'}
{'symbol': 'MCPH1', 'full_name': 'microcephalin 1'}
{'symbol': 'CEP63', 'full_name': 'centrosomal protein 63'}
{'symbol': 'ATRIP', 'full_name': 'ATR interacting protein'}
{'symbol': 'ASPM', 'full_name': 'abnormal spindle microtubule assembly'}
{'symbol': 'WDR62', 'full_name': 'WD repeat domain 62'}


#### Find all genes mentioned in this paper PMID= 20301772. 
The following genes has been linked to this paper: 
- ATR: ATR serine/threonine kinase
- CDK6: cyclin dependent kinase 6
- PHC1: 'polyhomeotic homolog 1
- RBBP8: RB binding protein 8, endonuclease
- STIL: STIL centriolar assembly protein
- CEP135: centrosomal protein 135
- CEP152: centrosomal protein 152
- NIN: ninein
- CDK5RAP2: CDK5 regulatory subunit associated protein 2
- CENPJ: centromere protein J
- KNL1: kinetochore scaffold 1
- MCPH1: microcephalin 1
- CEP63: centrosomal protein 63
- ATRIP: ATR interacting protein
- ASPM: abnormal spindle microtubule assembly
- WDR62: WD repeat domain 62


This is how i would design the relations between entities, and expose them to the customer:

| **Key Entity**   |  **AssociatedProperty**  |
| ----------- | ----------- |
| Gene | *hasGeneSymbol* |
| Gene | *hasGeneName* |
| Gene | *hasGeneSynonyms* |
| Gene | *hasChromosome* |
| Gene | *hasCytogeneticBand* |
| Gene | *hasOrganism* |
| Gene | *hasTaxon* |
| Gene | *hasRefSeqTranscriptID* |
| Gene | *hasRefSeqProteinID* |
| Gene | *hasCCDS* |
| Scientific Publication | *hasPMID* |
| Scientific Publication | *hasPublisher* |
| Scientific Publication | *hasJournal* |
| Scientific Publication | *hasPublicationYear* |
| Scientific Publication | *hasTitle* |
| Scientific Publication | *hasLanguage* |
| Scientific Publication | *hasPublicationType* |
| Scientific Publication | *hasAuthors* |

